### Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


### Load Run Data and Patient Selection

Installation tools for data acquisition

In [2]:
%%bash

# Entrez Direct
sh -c "$(curl -fsSL https://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh)"

# SRA Toolkit
wget https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/current/sratoolkit.current-ubuntu64.tar.gz
tar -xzf sratoolkit.current-ubuntu64.tar.gz



Entrez Direct has been successfully downloaded and installed.

In order to complete the configuration process, please execute the following:

  echo "export PATH=/root/edirect:\${PATH}" >> ${HOME}/.bashrc

or manually edit the PATH variable assignment in your .bashrc file.

Would you like to do that automatically now? [y/N]
Holding off, then.

To activate EDirect for this terminal session, please execute the following:

export PATH=${HOME}/edirect:${PATH}



--2025-12-24 05:46:47--  https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/current/sratoolkit.current-ubuntu64.tar.gz
Resolving ftp-trace.ncbi.nlm.nih.gov (ftp-trace.ncbi.nlm.nih.gov)... 130.14.250.31, 130.14.250.10, 130.14.250.11, ...
Connecting to ftp-trace.ncbi.nlm.nih.gov (ftp-trace.ncbi.nlm.nih.gov)|130.14.250.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 92046842 (88M) [application/x-gzip]
Saving to: ‘sratoolkit.current-ubuntu64.tar.gz’

     0K .......... .......... .......... .......... ..........  0%  142K 10m33s
    50K .......... .......... .......... .......... ..........  0% 1.35M 5m49s
   100K .......... .......... .......... .......... ..........  0%  356K 5m16s
   150K .......... .......... .......... .......... ..........  0% 36.6M 3m58s
   200K .......... .......... .......... .......... ..........  0%  286K 4m13s
   250K .......... .......... .......... .......... ..........  0% 56.2M 3m31s
   300K .......... .......... .......... .......... .

In [3]:
%%bash
# As asked during installation adding edirect to PATH
export PATH=${HOME}/edirect:${PATH}

In [4]:
%%bash
cd $HOME
tar -xzf ../content/sratoolkit.current-ubuntu64.tar.gz

Download RunInfo metadata for PRJNA714799 and patient selection

In [5]:
%%bash
$HOME/edirect/esearch -db sra -query PRJNA714799 | \
$HOME/edirect/efetch -format runinfo > PRJNA714799_runinfo.csv


In [6]:
%%bash
ls -lh

total 89M
drwx------ 5 root root 4.0K Dec 24 05:46 drive
-rw-r--r-- 1 root root 224K Dec 24 05:49 PRJNA714799_runinfo.csv
drwxr-xr-x 1 root root 4.0K Dec 11 14:34 sample_data
drwxrwxr-x 5 root root 4.0K Nov 28 23:10 sratoolkit.3.3.0-ubuntu64
-rw-r--r-- 1 root root  88M Nov 28 23:10 sratoolkit.current-ubuntu64.tar.gz


In [7]:
import pandas as pd

# Load the CSV file into a pandas DataFrame
df = pd.read_csv("PRJNA714799_runinfo.csv")
print(df.shape)
df.head(3)

(420, 47)


,Run,ReleaseDate,LoadDate,spots,bases,spots_with_mates,avgLength,size_MB,AssemblyName,download_path,...,Affection_Status,Analyte_Type,Histological_Type,Body_Site,CenterName,Submission,dbgap_study_accession,Consent,RunHash,ReadHash
0,SRR13974039,2022-04-18 00:00:24,2021-03-16 08:45:24,2132268,615326977,2053223,288,350,GCA_000001405.13,https://sra-downloadb.be-md.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,SEOUL NATIONAL UNIVERSITY,SRA1206760,NaN,public,25A5EE99E44E4FBC0D6B95BE1221A156,5A979126CE22F541C18C1B3DB7394495
1,SRR13974041,2022-04-18 00:00:24,2021-03-16 08:45:29,2704436,789609381,2632994,291,420,GCA_000001405.13,https://sra-downloadb.be-md.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,SEOUL NATIONAL UNIVERSITY,SRA1206760,NaN,public,099ADF641FDB3EC474D711584108DBB1,86C1109112E35E3F10C96AFAB8216EC2
2,SRR13974040,2022-04-18 00:00:24,2021-03-16 08:45:31,2263227,659576113,2218251,291,354,GCA_000001405.13,https://sra-downloadb.be-md.ncbi.nlm.nih.gov/s...,...,NaN,NaN,NaN,NaN,SEOUL NATIONAL UNIVERSITY,SRA1206760,NaN,public,C85AA55C37FB24B12E19C3B70DC7EC67,CBF81247DE6651D5F9633ACB7E61A8A4


⚠️**WARNING**

In **Supplementary Table 1** samples ID defined as CTDC01, CTDC01-2, CTDC01-5 or CTDC02, CTDC02-2, CTDC02-15 representing baseline, 1st response evaluation, disease progression (at PD) respectively. RunInfo table _PRJNA714799_runinfo.csv_ which contains info about patients and files uploaded to NCBI under BioProject PRJNA714799 **DOES NOT have such sample IDs**!

In [8]:
# Boolean mask: True where any cell contains "CTDC"
mask = df.apply(lambda col: col.astype(str).str.contains("CTDC", na=False))

# Rows where at least one column contains "CTDC"
result = df[mask.any(axis=1)]

print(result)

Empty DataFrame
Columns: [Run, ReleaseDate, LoadDate, spots, bases, spots_with_mates, avgLength, size_MB, AssemblyName, download_path, Experiment, LibraryName, LibraryStrategy, LibrarySelection, LibrarySource, LibraryLayout, InsertSize, InsertDev, Platform, Model, SRAStudy, BioProject, Study_Pubmed_id, ProjectID, Sample, BioSample, SampleType, TaxID, ScientificName, SampleName, g1k_pop_code, source, g1k_analysis_group, Subject_ID, Sex, Disease, Tumor, Affection_Status, Analyte_Type, Histological_Type, Body_Site, CenterName, Submission, dbgap_study_accession, Consent, RunHash, ReadHash]
Index: []

[0 rows x 47 columns]


Let's try to figure out what we have in this BioProject

In [9]:
# Count the number of unique values in each column and sort them in descending order
unique_counts = df.nunique().sort_values(ascending=False)
print(unique_counts)


Run                      420
bases                    420
spots                    420
spots_with_mates         420
LibraryName              420
download_path            420
Experiment               420
ReadHash                 420
Sample                   420
SampleName               420
RunHash                  420
BioSample                420
LoadDate                 360
size_MB                  353
avgLength                 70
ReleaseDate                8
LibrarySelection           1
LibraryStrategy            1
AssemblyName               1
LibrarySource              1
InsertSize                 1
Model                      1
LibraryLayout              1
SampleType                 1
BioProject                 1
SRAStudy                   1
ProjectID                  1
ScientificName             1
TaxID                      1
InsertDev                  1
Platform                   1
CenterName                 1
Submission                 1
Sex                        1
Tumor         

In [10]:
# Extract the unique values from the 'SampleName' column
unique_samples = df['SampleName'].unique()
print(unique_samples[0:10])
#print(unique_samples)

['CTC325' 'CTC325-2' 'CTC325-9' 'PBMC_C_fw012' 'PBMC_C_fw074' 'CTC320'
 'CTC320-2' 'CTC320-3' 'CTC321' 'CTC321-2']


In [11]:
# Create new column with only the part before dash
df['SampleBase'] = df['SampleName'].str.split('-').str[0]
print(df[['SampleName', 'SampleBase']].head(8))

     SampleName    SampleBase
0        CTC325        CTC325
1      CTC325-2        CTC325
2      CTC325-9        CTC325
3  PBMC_C_fw012  PBMC_C_fw012
4  PBMC_C_fw074  PBMC_C_fw074
5        CTC320        CTC320
6      CTC320-2        CTC320
7      CTC320-3        CTC320


In [12]:
df.loc[df["SampleBase"].str.contains("CTC", na=False), "SampleBase"].nunique()


165

In [13]:
len(df.loc[df["SampleBase"].str.startswith("CTC", na=False), "SampleBase"].unique())


76

In [14]:
# Just for playing with data let's take the first sample and filter rows for CTC325
df_ctc325 = df[df['SampleBase'] == 'CTC325']
print(df_ctc325[["Sample", "SampleName", "BioSample", "SampleBase", "Experiment"]])

       Sample SampleName     BioSample SampleBase   Experiment
0  SRS8470628     CTC325  SAMN18318191     CTC325  SRX10351781
1  SRS8470626   CTC325-2  SAMN18318189     CTC325  SRX10351779
2  SRS8470627   CTC325-9  SAMN18318190     CTC325  SRX10351780


Select one patient with multiple timepoints and save SRR IDs.

In [15]:
patient = "CTC325"
df_patient = df[df["SampleBase"] == patient]
df_patient[["Run", "SampleName", "SampleBase"]]

,Run,SampleName,SampleBase
0,SRR13974039,CTC325,CTC325
1,SRR13974041,CTC325-2,CTC325
2,SRR13974040,CTC325-9,CTC325


In [16]:
#Save SRR IDs to a text file
srr_list = df_patient["Run"].tolist()
with open("srr_list.txt", "w") as f:
    for srr in srr_list:
        f.write(srr + "\n")
srr_list

['SRR13974039', 'SRR13974041', 'SRR13974040']

### Download FASTQ files for analysis

In [17]:
%%bash

mkdir -p fastq/CTC325
cd fastq/CTC325

while read srr; do
    echo "Downloading $srr"
    $HOME/sratoolkit.*/bin/fasterq-dump --split-files --threads 8 $srr
done < ../../srr_list.txt

spots read      : 2,132,268
reads read      : 4,264,536
reads written   : 4,185,491
spots read      : 2,704,436
reads read      : 5,408,872
reads written   : 5,337,430
spots read      : 2,263,227
reads read      : 4,526,454
reads written   : 4,481,478


Copy the FASTQ folder to Google Drive to have an access to them later

In [18]:
%%bash
mkdir -p /content/drive/MyDrive/ColabFiles/GTMolecular/CTC325
cp -r /content/fastq/CTC325/* /content/drive/MyDrive/ColabFiles/GTMolecular/CTC325/

## After restarting we can start from here

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
import os

fastq_dir = '/content/drive/MyDrive/ColabFiles/GTMolecular/CTC325'
fastq_files = os.listdir(fastq_dir)
print(fastq_files)

['SRR13974039_1.fastq', 'SRR13974039_2.fastq', 'SRR13974040_1.fastq', 'SRR13974040_2.fastq', 'SRR13974041_1.fastq', 'SRR13974041_2.fastq']


### FastQC

In [23]:
%%bash

# Install FastQC
apt-get update
apt-get install -y fastqc

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,860 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,633 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [6,411 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,572 kB]
Get:14 http://s

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [24]:
%%bash
fastqc --version

FastQC v0.11.9


In [25]:
%%bash

# Make a folder for QC results
mkdir -p /content/results/qc

# Run FastQC on all files
fastqc -o /content/results/qc /content/drive/MyDrive/ColabFiles/GTMolecular/CTC325/*.fastq

Analysis complete for SRR13974039_1.fastq
Analysis complete for SRR13974039_2.fastq
Analysis complete for SRR13974040_1.fastq
Analysis complete for SRR13974040_2.fastq
Analysis complete for SRR13974041_1.fastq
Analysis complete for SRR13974041_2.fastq


Started analysis of SRR13974039_1.fastq
Approx 5% complete for SRR13974039_1.fastq
Approx 10% complete for SRR13974039_1.fastq
Approx 15% complete for SRR13974039_1.fastq
Approx 20% complete for SRR13974039_1.fastq
Approx 25% complete for SRR13974039_1.fastq
Approx 30% complete for SRR13974039_1.fastq
Approx 35% complete for SRR13974039_1.fastq
Approx 40% complete for SRR13974039_1.fastq
Approx 45% complete for SRR13974039_1.fastq
Approx 50% complete for SRR13974039_1.fastq
Approx 55% complete for SRR13974039_1.fastq
Approx 60% complete for SRR13974039_1.fastq
Approx 65% complete for SRR13974039_1.fastq
Approx 70% complete for SRR13974039_1.fastq
Approx 75% complete for SRR13974039_1.fastq
Approx 80% complete for SRR13974039_1.fastq
Approx 85% complete for SRR13974039_1.fastq
Approx 90% complete for SRR13974039_1.fastq
Approx 95% complete for SRR13974039_1.fastq
Started analysis of SRR13974039_2.fastq
Approx 5% complete for SRR13974039_2.fastq
Approx 10% complete for SRR13974039_2.fast

In [26]:
%%bash
# Install MultiQC via pip
pip install multiqc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.6/39.6 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 116.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.2 MB/s eta 0:00:00


In [27]:
%%bash

multiqc /content/results/qc/ -o /content/results/multiqc


/// MultiQC 🎄 v1.33

       file_search | Search path: /content/results/qc

            fastqc | Found 6 reports

     write_results | Data        : results/multiqc/multiqc_data
     write_results | Report      : results/multiqc/multiqc_report.html
           multiqc | MultiQC complete


In [28]:
%%bash

cp -r /content/results/multiqc/multiqc_report.html /content/drive/MyDrive/ColabFiles/GTMolecular/CTC325/

FastQC results represent good quality of data, multiqc summary saved on Google Drive and on GitHub.

### Alignment - not recommended to run in notebook! 
   - Alignment to GRCh38 reference genome on local machine
   - Sorting, indexing, and duplicate handling


NCBI Genome Assembly — GRCh38 can be downloaded from NCBI Datasets:
https://www.ncbi.nlm.nih.gov/assembly/GCF_000001405.40/


Also, reference human genome GRCh38 and correcponding indexes can be downloaded from https://gdc.cancer.gov/about-data/gdc-data-processing/gdc-reference-files?utm_source=chatgpt.com
